In [ ]:
%matplotlib inline

import numpy as np
from matplotlib import pyplot as plt
import timeit

import sys
from snl.inference.nde import SequentialNeuralLikelihood
import snl.simulators.lotka_volterra as sim_lv
from snl.ml.models.mafs import ConditionalMaskedAutoregressiveFlow

from delfi.utils.viz import plot_pdf
import delfi.inference as infer
import delfi.distribution as dd

from lfimodels.snl_exps.util import save_results, load_results
from lfimodels.snl_exps.util import init_g_lv as init_g
from lfimodels.snl_exps.util import load_setup_lv as load_setup
from lfimodels.snl_exps.util import load_gt_lv as load_gt
from lfimodels.snl_exps.util import calc_all_lprob_errs



seed = 42

model_id = 'mg1'
save_path = 'results/' + model_id
exp_id = 'seed'+str(seed)


# simulation setup
setup_dict = load_setup()

pars_true, obs_stats = load_gt(generator=init_g(seed=seed))
print('pars_true : ', pars_true)
print('obs_stats : ', obs_stats)


model = sim_lv.Model()
prior = sim_lv.Prior()
stats = sim_lv.Stats()

sim_model = lambda ps, rng: stats.calc(model.sim(ps, rng=rng)) 
# to run without summary stats: sim_model = model.sim !

# fit SNL


# * big todo: get whitening to work ! *


In [ ]:
infer = SequentialNeuralLikelihood(prior=prior, sim_model=sim_model)

# control MAF seed
rng = np.random
rng.seed(seed)

maf = ConditionalMaskedAutoregressiveFlow(n_inputs=prior.n_dims, 
                                          n_outputs=obs_stats.size, 
                                          n_hiddens=setup_dict['n_hiddens'], 
                                          act_fun=setup_dict['act_fun'], 
                                          n_mades=setup_dict['n_mades'], 
                                          
                                          batch_norm=False,           # these differ for 
                                          output_order='sequential', # the usage of our 
                                          mode='sequential',         # MAFs...
                                          
                                          input=None, 
                                          output=None, rng=rng)


# control sampler seed
rng = np.random
rng.seed(seed+1)

t = timeit.time.time()

learned_model = infer.learn_likelihood(obs_xs=obs_stats.flatten(), 
                       model=maf, 
                       n_samples=setup_dict['n_train'], 
                       n_rounds=setup_dict['n_rounds'],
                       train_on_all=setup_dict['train_on_all'],
                       thin=10, 
                       save_models=False, 
                       logger=sys.stdout, 
                       rng=rng)

print(timeit.time.time() -  t)


# check learned posterior

In [ ]:
from snl.util.plot import plot_hist_marginals
import snl.inference.mcmc as mcmc

thin = 10

log_posterior = lambda t: learned_model.eval([t, obs_stats.flatten()]) + prior.eval(t)
sampler = mcmc.SliceSampler(prior.gen(), log_posterior, thin=thin)

sampler.gen(max(200 // thin, 1), rng=rng)  # burn in

samples = sampler.gen(1000)

fig = plot_hist_marginals(data=samples, lims=[-5,2])
fig.set_figwidth(16)
fig.set_figheight(16)
fig.show()

### zoom-in

In [ ]:
fig = plot_hist_marginals(data=samples)
fig.set_figwidth(16)
fig.set_figheight(16)
fig.show()

# check conditional p(x|theta*)

In [ ]:
from delfi.utils.delfi2snl import MAFconditional
from delfi.distribution.PointMass import PointMass

samples_x = learned_model.gen(x=pars_true.flatten(), 
                              n_samples=1000)

fig = plot_hist_marginals(data=samples_x)
fig.set_figwidth(16)
fig.set_figheight(16)
fig.show()